# Output the Association rule

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load dataset
df = pd.read_csv('./transactions.csv')

# Preprocess data
transactions = df['items'].apply(lambda x: x.split(', ')).tolist()
encoder = TransactionEncoder()
onehot = encoder.fit(transactions).transform(transactions)
onehot_df = pd.DataFrame(onehot, columns=encoder.columns_)

# Apply FP-Growth algorithm
# Generate frequent itemsets with a lower support threshold
min_support = 0.05  # Adjust this threshold based on your needs
frequent_itemsets = fpgrowth(onehot_df, min_support=min_support, use_colnames=True)

# Display frequent itemsets
print(frequent_itemsets)

# Generate association rules
num_itemsets = len(frequent_itemsets)
# Generate association rules with a lower confidence threshold
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="confidence", min_threshold=0.3)  # Adjust as needed

# Display results
print("Association Rules:")
print(rules)

# Save frequent itemsets and association rules to CSV files
#frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)
rules.to_csv('fpgrowth_association_rules.csv', index=False)

print("Frequent itemsets and association rules have been saved to 'frequent_itemsets.csv' and 'association_rules.csv'.")

# Association Rule Score Function

In [ ]:
import pandas as pd

def find_suitable_foods(food_types):
    """Find suitable food types based on the input list of food types."""
    global confidence_dishes_df

    # Load the association rules from CSV
    #rules_df = pd.read_csv("./apriori_association_rules.csv")
    rules_df = pd.read_csv("./fpgrowth_association_rules.csv")

    # Filter rules where any of the input food types are in the antecedents
    suitable_rules = rules_df[rules_df['antecedents'].apply(lambda x: any(food in eval(x) for food in food_types))]

    # Prepare output as a list of tuples (food, score)
    suitable_foods = []
    for index, row in suitable_rules.iterrows():
        for consequent in eval(row['consequents']):
            # Append the consequent and its confidence score
            suitable_foods.append((consequent, row['confidence']))

    confidence_dishes_df = pd.DataFrame(suitable_foods, columns=['Type', 'Score'])
    # Sort by 'Combined_Score' in descending order
    confidence_dishes_df = confidence_dishes_df.sort_values(by='Score', ascending=False)

# Example usage
if __name__ == "__main__":
    # Input food types
    input_foods = ["Sandwich"]  # Change this to any list of food types you want to check
    # Find suitable foods
    find_suitable_foods(input_foods)
    print(confidence_dishes_df)

# Recommender System Score

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

def recommend_dish(prep_time, cook_time):
    global recommend_df

    # Load the dataset
    df = pd.read_csv('./data_use_to_test_the_recommender_system/indian_food.csv')

    # Fill any NaNs with zeros
    df['prep_time'] = df['prep_time'].fillna(0)
    df['cook_time'] = df['cook_time'].fillna(0)

    # Prepare the feature matrix for prep_time and cook_time
    features = df[['prep_time', 'cook_time']]
    # Normalize the features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    # Create an input array for the provided times
    input_times = [[prep_time, cook_time]]

    # Scale the input times using the same scaler
    scaled_input = scaler.transform(input_times)

    # Compute the similarity scores using cosine similarity
    sim_scores = cosine_similarity(scaled_input, scaled_features)

    # Get indices of all dishes
    sim_scores = sim_scores.flatten()

    # Create a DataFrame for recommendations with scores
    recommend_df = pd.DataFrame({
        'Dish': df['name'],
        'Type': df['type'],
        'Score': sim_scores
    })

    # Sort by score in descending order and get top 5 recommendations
    recommend_df = recommend_df.sort_values(by='Score', ascending=False)


# Example usage
prep_time_input = 20
cook_time_input = 30

recommend_dish(prep_time_input, cook_time_input)
print(recommend_df)

# Final Score and Recommend

In [ ]:
import pandas as pd

def combined_score(prep_time_input, cook_time_input, food_type):
    global combined_df

    Recommander_Weight = 1.0 # Weight of Content-base Recommander
    Association_Weight = 1.0 # Weight of Association Rule

    find_suitable_foods(food_type)
    # find_similar_dishes(prep_time_input, cook_time_input)
    recommend_dish(prep_time_input, cook_time_input)

    # Merge the dataframes on the 'Type' column
    merged_df = recommend_df.merge(confidence_dishes_df, on='Type', how='left')

    # Add the scores from the two dataframes
    combined_df = merged_df[['Dish', 'Type', 'Score_x', 'Score_y']].rename(columns={'Score_x': 'Recommander_Score', 'Score_y': 'Confidence_Score'})
    combined_df['Combined_Score'] = (combined_df['Recommander_Score'] * Recommander_Weight) + (combined_df['Confidence_Score'] * Association_Weight)

    # Drop rows with NaN values in 'Combined_Score'
    combined_df = combined_df.dropna()

    # Sort by 'Combined_Score' in descending order
    sorted_combined_df = combined_df.sort_values(by='Combined_Score', ascending=False)

    # Display the sorted combined dataframe
    print(sorted_combined_df)

prep_time_input = 10
cook_time_input = 20
food_type_input = ["Meat","Salad"]
combined_score(prep_time_input, cook_time_input, food_type_input)
